In [54]:
#!/bin/bash
""" " 2>/dev/null 
# Run python in inf loop
while true; do python3 $0 "${@}"; sleep 0.1; done
" """

#requirements psutils termcolor tqdm
#apt-get install ntfs-3g gnome-terminal

import drive_utils
import termwriter 
import filelogger
import misc
import copier
from termwriter import *

import os
#from termcolor import colored as col
from tqdm import tqdm
import shutil
import time
from datetime import datetime

if misc.jupyter_mode():
    tqdm.monitor_interval = 0
    get_ipython().run_line_magic('reload_ext', 'autoreload')
    get_ipython().run_line_magic('autoreload', '2')
    misc.save_jupyter_as_bash('media-copier.ipynb')

In [ ]:
vid_ext = misc.get_video_ext()
poll = drive_utils.drivePollster()

status = misc.dotdict(
    terminated = False,
    uncopied = 0)

countdown    = 1

for ii in range(1000):
     
    #--------------------------------------------------------
    #Get all the drive logic ready
    drives = drive_utils.get_drives()

    #Use largest USB drive as harddrive
    hdd = None
    for drive in drives:
        if drive_utils.is_usb_drive(drive.device):
            if hdd is None:
                hdd = drive
            elif drive.size_total > hdd.size_total:
                hdd = drive

    #Use smaller USB drive as SSD
    ssd = None
    for i in drives:
        if (drive_utils.is_usb_drive(i.device) and
            not i==hdd):
            ssd = i
    
    #--------------------------------------------------------
    #Triggers: when devices are plugged in/out
    if poll.did_change() or ii==0: 
        status.terminated = False
        status.uncopied = 0
        start = time.time()
    
    #--------------------------------------------------------
    # Crude color display
    c = coldict
    #HEADER OKBLUE OKGREEN WARNING FAIL ENDC BOLD UNDERLINE
    
    termwriter.cls()
    cols, lines = shutil.get_terminal_size((80, 20))
    if not (hdd and ssd):
        printterm(c.RED, end='')
        writeterm("-"*cols, '+', '+')
        writeterm('', '|', '|')
        writeterm(c.RED+' Please insert' +c.BLUE+' USB Hard drive' +c.RED+' and' +c.GREEN+' Camera SD' +c.RED +'  '
                        +'.'*(ii%4+1), '|','|')
        writeterm('', '|', '|')
        writeterm("-"*cols, '+', '+')
        printterm(c.END, end='')

    else:
        #--------------------------------------------------------
        # Copy from SSD to hard-drive
        printterm(c.BLUE, end='')
        writeterm("-"*cols, '+', '+')
        writeterm(' ---- Copy from: ----', '|','|')
        writeterm('   '+ssd.mountpoint, '|','|')
        writeterm("    free:  %d GB"% (ssd.size_free //(2**30)), '|','|')
        writeterm("    total: %d GB"%(ssd.size_total//(2**30)), '|','|')
        writeterm("-"*cols, '+', '+')
        printterm(c.GREEN, end='')
        writeterm(' ---- Copy to: ----', '|','|')
        writeterm('   '+hdd.mountpoint, '|','|')
        writeterm("     free:  %d GB"% (hdd.size_free //(2**30)), '|','|')
        writeterm("     total: %d GB"%(hdd.size_total//(2**30)), '|','|')
        writeterm("-"*cols, '+', '+')
        printterm(c.END, end='')
        
        #Do not copy if the routine is already finished
        if status.terminated:
            printterm()
            if status.uncopied > 0:
                printterm("SD card %.3fGB uncopied! Drive too full!"%(status.uncopied/2**30))
                printterm("Replace harddrive...")
            else:
                printterm("SD card backup is complete!")
                printterm("You may remove devices...")
            continue
        
        #--------------------------------------------------------
        # Wait 20 second
        printterm()
        if (time.time()-start) < countdown:
            printterm(' %d s left to cancel (plug out device)...'%(
                countdown - (time.time()-start)))
            continue
            
        #--------------------------------------------------------
        # Do the copy
        printterm(" Scanning files:")

        hdd_dir = os.path.join(hdd.mountpoint, 'SD_autobackup')

        out_dir = os.path.join(hdd_dir,
                               datetime.now().strftime('%Y%m%d-')+
                               copier.get_valid_filename(ssd.id)+'--%.1fGB'%(
                                   ssd.size_total/(2**30)))


        os.makedirs(hdd_dir, exist_ok=True)

        fileregister = filelogger.get_fileregister(hdd_dir)
        newregister = {}


        t = tqdm(total=ssd.size_used, file=sysout, unit='B', unit_scale=True)
        #force full path with trailing /
        sd_base = os.path.abspath(os.path.join(ssd.mountpoint,''))
        totsize = 0
        filecount = misc.dotdict(files=0, size=0, vidfiles=0, vidsize=0)
        
        for _, dirs, files in os.walk(sd_base, followlinks=False):
            for file in files:
                fpath = os.path.join(_, file)
                try:
                    key, attrs = filelogger.to_register(fpath)
                except FileNotFoundError:
                    pass
                    #print('File not found ', fpath)
                else:
                    if key not in fileregister:
                        newregister[key] = attrs

                        filecount.files += 1
                        filecount.size  += attrs.size
                        if os.path.splitext(attrs.path)[-1] in vid_ext:
                            filecount.vidfiles += 1
                            filecount.vidsize  += attrs.size

                t.update(attrs.size)
        t.close()
        printterm('\r', end='')

        available_space = hdd.size_free*0.9999 #leave 100kB tolerance for every 1GB
        
        printterm('Uncopied videos %.2fGB; %d files.'%(filecount.vidsize/2**30, filecount.vidfiles))
        printterm('Uncopied others %.2fGB; %d files.'%(filecount.size/2**30, filecount.files))
        
        #Determine max space that can be occupied
        if filecount.size > available_space:
            size_fill = 0
            size_left = 0
            for key, attrs in newregister.items():
                if size_fill + attrs.size < available_space:
                    size_fill += attrs.size
                else:
                    size_left += attrs.size
            
            printterm('Warning!!! Full drive; %.3fGB files abandoned.'%(
                (size_left)/2**30)+'%')
            
        printterm()

        i=0
        t = tqdm(total=filecount.size, file=sysout, unit='B', unit_scale=True)
        for key, attrs in newregister.items():
            i+=1

            src = attrs.path
                
            available_space -= attrs.size
            if available_space<0:
                status.uncopied += attrs.size
                continue

            # path.join breaks think it happens with files starting with a dot
            dest = os.path.abspath(out_dir +'/'+src[len(sd_base):])
            dest_dir = os.path.dirname(dest)

            if not os.path.isdir(dest_dir):
                os.makedirs(dest_dir)

            keeps = misc.dotdict()
            for chunksz in copier.copy_with_progress(src, dest, keeps=keeps):
                t.update(chunksz)


            if copier.hash_equals(keeps.md5, copier.hash_file(dest)):
                filelogger.update_fileregister(hdd_dir, attrs)
                

        t.close()
        printterm('\r', end='')
        
        #misc.systxt(['eject', hdd.mountpoint])
        #misc.systxt(['eject', ssd.mountpoint])
        status.terminated = True

        
    time.sleep(0.5)